In [4]:
import pandas as pd
import numpy as np
from scipy.stats import mannwhitneyu

In [6]:
dataA = pd.read_csv("RDW.csv").drop(columns=["ID"])      
dataB = pd.read_csv("JS.csv").drop(columns=["ID"]) 

# print(dataA.head())
# print(dataB.head())